In [1]:
import requests
from bs4 import BeautifulSoup
import json

/Users/Welcome/Library/Python/3.9/lib/python/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [25]:
def makeURL(search = "", period="", start_date="", end_date="") -> str:
    url = "https://search.daum.net/search?w=news&DA=STC&enc=utf8"
    
    if period != "":
        period_url = "&period=%s" % (period)
        
        elif period == "u": #기간 조정
            period_url = period_url + "&sd=%s&ed=%s" % (start_date, end_date)
    
    if search !="":
        search_url = "&q=%s" % (search)
    
    return url

def newsScrape(search):
    URL = makeURL(search)
    original_html = requests.get(URL)
    html = BeautifulSoup(original_html.text, "html.parser")
    title_list = html.select("div.cont_divider > ul.list_news > li div.wrap_cont > a")
    #content_list = html.select("div.cont_divider > ul.list_news > li div.wrap_cont > p")

    for i in title_list:
        print(i.get_text())



#search 없을때 날짜로만 데이터 수집할때는
url = "https://news.daum.net/breakingnews/"

"반려견 이름 연우"..김연아♥고우림 결혼, 팬들은 진작 알고 있었다
김연아-고우림 "확고한 믿음" 10월 결혼할 결심[종합]
김연아, 성악가 고우림씨와 결혼
[종합]김연아, '품절녀 곰신' 예약..♥고우림과 3년 열애→10월 결혼
김연아 결혼발표, 고우림은 누구?
'피겨 여왕' 김연아, 성악가 고우림과 10월 결혼
'피겨여왕' 김연아 결혼 소식에 일본 언론도 '들썩'
'김연아 시아버지' 되는 고경수 목사 "예비 며느리는 착한 사람"
'피겨 퀸' 김연아, 포레스텔라 고우림과 10월 결혼
김연아·고우림 결혼에 아시아도 들썩..화제의 중심
